In [1]:
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import patsy
import sklearn.metrics as metrics
import statsmodels.formula.api as smf
import statsmodels.api as sm
from plotnine import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import (
    LinearRegression,
    LogisticRegression,
    LogisticRegressionCV,
)
from sklearn.metrics import (
    auc,
    brier_score_loss,
    confusion_matrix,
    mean_squared_error,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.tools.eval_measures import rmse

warnings.filterwarnings("ignore")

In [2]:
def regression_results(y_true, y_pred):
    
    # Regression metrics
    explained_variance = metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    median_absolute_error = metrics.median_absolute_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)

    print("explained_variance: ", round(explained_variance, 4))
    print("r2: ", round(r2, 4))
    print("MAE: ", round(mean_absolute_error, 4))
    print("MSE: ", round(mse, 4))
    print("RMSE: ", round(np.sqrt(mse), 4))


def coef_matrix(X, model):

    coef_matrix = pd.concat(
        [pd.DataFrame(X.columns), pd.DataFrame(np.transpose(model.coef_))], axis=1
    )
    coef_matrix.columns = ["variable", "coefficient"]
    coef_matrix = coef_matrix.append(
        {"variable": "Intercept", "coefficient": model.intercept_},
        ignore_index=True,
    )
    return coef_matrix


def cv_summary(lambdas, C_values, model):
    d = {
        "lambdas": lambdas,
        "C_values": C_values,
        "mean_cv_score": model.scores_[1].mean(axis=0),
    }
    return pd.DataFrame(data=d)


def create_roc_plot(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    all_coords = pd.DataFrame({"fpr": fpr, "tpr": tpr, "thresholds": thresholds})

    plot = (
        ggplot(all_coords, aes(x="fpr", y="tpr"))
        + geom_line(color=color[0], size=0.7)
        + geom_area(position="identity", fill="mediumaquamarine", alpha=0.3)
        + xlab("False Positive Rate (1-Specifity)")
        + ylab("True Positive Rate (Sensitivity)")
        + geom_abline(intercept=0, slope=1, linetype="dotted", color="black")
        + scale_y_continuous(limits=(0, 1), breaks=seq(0, 1, 0.1), expand=(0, 0.01))
        + scale_x_continuous(limits=(0, 1), breaks=seq(0, 1, 0.1), expand=(0.01, 0))
        + theme_bw()
    )
    return plot


def sigmoid_array(x):
    return 1 / (1 + np.exp(-x))


def generate_fold_prediction(model, X, fold, param_index):
    fold_coef = model.coefs_paths_[1][fold, param_index, :]
    return sigmoid_array(
        np.dot(X, np.transpose(fold_coef)[:-1]) + np.transpose(fold_coef)[-1]
    )


def create_loss_plot(all_coords, optimal_threshold, curr_exp_loss):
    all_coords_copy = all_coords.copy()
    all_coords_copy["loss"] = (
        all_coords_copy.false_pos * FP + all_coords_copy.false_neg * FN
    ) / all_coords_copy.n

    t = optimal_threshold
    l = curr_exp_loss

    plot = (
        ggplot(all_coords_copy, aes(x="thresholds", y="loss"))
        + geom_line(color=color[0], size=0.7)
        + scale_x_continuous(breaks=seq(0, 1.1, by=0.1))
        + coord_cartesian(xlim=(0, 1))
        + geom_vline(xintercept=t, color=color[0])
        + annotate(
            geom="text",
            x=t - 0.01,
            y=max(all_coords_copy.loss) - 0.4,
            label="best threshold: " + str(round(t, 2)),
            colour=color[1],
            angle=90,
            size=7,
        )
        + annotate(geom="text", x=t + 0.06, y=l, label=str(round(l, 2)), size=7)
        + theme_bw()
    )
    return plot


def create_roc_plot_with_optimal(all_coords, optimal_threshold):
    all_coords_copy = all_coords.copy()
    all_coords_copy["sp"] = 1 - all_coords_copy.true_neg / all_coords_copy.neg
    all_coords_copy["se"] = all_coords_copy.true_pos / all_coords_copy.pos

    best_coords = all_coords_copy[all_coords_copy.thresholds == optimal_threshold]
    sp = best_coords.sp.values[0]
    se = best_coords.se.values[0]

    plot = (
        ggplot(all_coords_copy, aes(x="sp", y="se"))
        + geom_line(color=color[0], size=0.7)
        + scale_y_continuous(breaks=seq(0, 1.1, by=0.1))
        + scale_x_continuous(breaks=seq(0, 1.1, by=0.1))
        + geom_point(data=pd.DataFrame({"sp": [sp], "se": [se]}))
        + annotate(
            geom="text",
            x=sp,
            y=se + 0.03,
            label=str(round(sp, 2)) + ", " + str(round(se, 2)),
            size=7,
        )
        + geom_area(position="identity", fill="mediumaquamarine", alpha=0.3)
        + xlab("False Positive Rate (1-Specifity)")
        + ylab("True Positive Rate (Sensitivity)")
        + geom_abline(intercept=0, slope=1, linetype="dotted", color="black")
        + theme_bw()
    )
    return plot

In [3]:
data = pd.read_csv("/Users/jacopobinati/Desktop/Prediction ML/assignment 3/datafinal2.csv")
data

,year,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,default_f,sales_mil_log_sq,flag_low_d1_sales_mil_log,flag_high_d1_sales_mil_log,d1_sales_mil_log_mod,d1_sales_mil_log_mod_sq,ebtda,t2_ebtda,ebtda_to_assets,high_growth
0,2010,1.002029e+06,2010-01-01,2010-12-31,22114.814453,230781.484375,202551.859375,0.0,0.000000,0.000000,...,no_default,0.028856,0,0,0.000000,0.000000,58244.443359,26748.148438,-0.113955,False
1,2010,1.003200e+06,2010-01-01,2010-12-31,74.074074,922.222229,5388.888672,0.0,0.000000,0.000000,...,no_default,18.349431,0,0,0.000000,0.000000,359.259254,-2311.111183,-1.427723,False
2,2010,1.011889e+06,2010-01-01,2010-12-31,27514.814453,105433.343750,6611.111328,0.0,0.000000,0.000000,...,no_default,1.154608,0,0,0.000000,0.000000,72477.779297,133377.777344,0.066279,True
3,2010,1.014183e+06,2010-01-01,2010-12-31,10281.481445,149503.703125,3622.222168,0.0,0.000000,0.000000,...,no_default,4.979001,0,0,0.000000,0.000000,14403.703613,11114.814517,-0.011698,False
4,2010,1.022796e+06,2010-01-01,2010-12-31,1537.036987,8192.592773,1577.777832,0.0,0.000000,0.000000,...,no_default,12.497002,0,0,0.000000,0.000000,-7992.592896,-574.074097,0.502257,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57080,2013,4.633865e+11,2013-01-01,2013-12-31,1722.222168,3325.926025,337.037048,0.0,0.000000,0.000000,...,no_default,15.463885,0,0,-0.147677,0.021808,1899.999939,4018.518491,0.046763,False
57081,2013,4.635828e+11,2013-01-01,2013-12-31,44.444443,1522.222168,166.666672,0.0,185.185181,185.185181,...,no_default,23.525539,0,0,0.060980,0.003719,1733.333359,-5944.444336,-3.300955,False
57082,2013,4.637275e+11,2013-01-01,2013-12-31,0.000000,9311.111328,211.111115,0.0,0.000000,0.000000,...,no_default,28.580871,0,0,0.000000,0.000000,-2007.407349,-7188.888916,-0.556484,False
57083,2013,4.639820e+11,2013-01-01,2013-12-31,12003.704102,109114.812500,150618.515625,0.0,0.000000,0.000000,...,no_default,0.987503,0,0,0.662642,0.439095,52311.110352,24096.295898,-0.128010,False


In [4]:
data.groupby('year')['high_growth'].value_counts()

year  high_growth
2010  False           9507
      True            4077
2011  False           9963
      True            4273
2012  False          10300
      True            4418
2013  False          10180
      True            4367
Name: count, dtype: int64

In [5]:
X = data.drop(columns=['high_growth'])
y = data['high_growth']

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Checking the stability of flagged observations in train and holdout sets
flagged_train_ratio = (y_train == 1).sum() / len(y_train)
flagged_holdout_ratio = (y_holdout == 1).sum() / len(y_holdout)

In [6]:
share_train = np.sum(y_train == 'yes') / len(y_train)
share_holdout = np.sum(y_holdout == 'yes') / len(y_holdout)

print("Share of flagged observations in train set:", share_train)
print("Share of flagged observations in holdout set:", share_holdout)

Share of flagged observations in train set: 0.0
Share of flagged observations in holdout set: 0.0


In [7]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)


In [8]:
data1 = data

In [9]:
_pl = data1.filter(like='ebtda')
sales_list = list(_pl)
sales_list

['ebtda', 't2_ebtda', 'ebtda_to_assets']

In [10]:
P1 = ['age', 'age2', 'ind', 'ind2_cat', 'foreign_management', 'urban_m', 'ebtda', "ebtda_to_assets","female", "ceo_age", "ceo_count", "flag_low_ceo_age", "flag_high_ceo_age", "flag_miss_ceo_age", "ceo_young"]
P2 = ['extra_exp_pl','extra_inc_pl','extra_profit_loss_pl','inc_bef_tax_pl','inventories_pl','material_exp_pl','profit_loss_year_pl',
 'personnel_exp_pl','extra_exp_pl_flag_high',
 'extra_inc_pl_flag_high',
 'inventories_pl_flag_high',
 'material_exp_pl_flag_high',
 'personnel_exp_pl_flag_high',
 'extra_exp_pl_flag_error',
 'extra_inc_pl_flag_error',
 'inventories_pl_flag_error',
 'material_exp_pl_flag_error',
 'personnel_exp_pl_flag_error',
 'extra_profit_loss_pl_flag_low',
 'inc_bef_tax_pl_flag_low',
 'profit_loss_year_pl_flag_low',
 'extra_profit_loss_pl_flag_high',
 'inc_bef_tax_pl_flag_high',
 'profit_loss_year_pl_flag_high',
 'extra_profit_loss_pl_flag_zero',
 'inc_bef_tax_pl_flag_zero',
 'profit_loss_year_pl_flag_zero',
 'extra_profit_loss_pl_quad',
 'inc_bef_tax_pl_quad',
 'profit_loss_year_pl_quad']

P3 = ['total_assets_bs',
 'intang_assets_bs',
 'curr_liab_bs',
 'fixed_assets_bs',
 'liq_assets_bs',
 'curr_assets_bs',
 'share_eq_bs',
 'subscribed_cap_bs',
 'tang_assets_bs',
 'curr_liab_bs_flag_high',
 'liq_assets_bs_flag_high',
 'subscribed_cap_bs_flag_high',
 'curr_liab_bs_flag_error',
 'liq_assets_bs_flag_error',
 'subscribed_cap_bs_flag_error',
 'share_eq_bs_flag_low',
 'share_eq_bs_flag_high',
 'share_eq_bs_flag_zero',
 'share_eq_bs_quad','ebtda', 't2_ebtda', 'ebtda_to_assets']

P4 = ["sales", "ln_sales", "sales_mil", "sales_mil_log", "d1_sales_mil_log", "sales_mil_log_sq", "flag_low_d1_sales_mil_log",
        "flag_high_d1_sales_mil_log", "d1_sales_mil_log_mod", "d1_sales_mil_log_mod_sq"] 
P5 = ["balsheet_flag", "balsheet_length", "balsheet_notfullyear"]
rawvars = [
    "curr_assets",
    "curr_liab",
    "extra_exp",
    "extra_inc",
    "extra_profit_loss",
    "fixed_assets",
    "inc_bef_tax",
    "intang_assets",
    "inventories",
    "liq_assets",
    "material_exp",
    "personnel_exp",
    "profit_loss_year",
    "sales",
    "share_eq",
    "subscribed_cap",
]

In [11]:
engvar3 = []
for col in data.columns:
    if (
        col.endswith("flag_low")
        or col.endswith("flag_high")
        or col.endswith("flag_error")
        or col.endswith("flag_zero")
    ):
        engvar3.append(col)

In [12]:
interactions1 = [
    "sales*age",
    "sales*age2",
    "curr_assets_bs*d1_sales_mil_log_mod",
    "ceo_young*sales_mil_log",
    "ebtda*ceo_age",
    "ebtda*foreign_management",
    "ebtda*female",
    "ebtda*labor_avg_mod",
]
interactions2 = [
    "ebtda_to_assets*age",
    "ebtda_to_assets*female",
    "ebtda_to_assets*profit_loss_year_pl",
    "ebtda_to_assets*foreign_management",
]

In [18]:
variables = P1 + P2 + P3 + P4 + P5


In [19]:
model1 = P1 + P2 + P3
model2 = P1 + P2 + P3 + P4
model3 = P1 + P2 + P3 + P4 + P5 #+ interactions1 + interactions2

## checking for duplicates

In [22]:
missing_columns = set(model3) - set(data1.columns)
if missing_columns:
    print("The following columns in model3 are missing in data1:")
    for column in missing_columns:
        print(column)
else:
    print("All columns in model3 are present in data1.")


All columns in model3 are present in data1.


## LOGIT 

In [20]:
from sklearn.model_selection import cross_val_score
rmse_list = []

for model_name, features in zip(['model1', 'model2', 'model3'], [model1, model2, model3]):
    X = data1[features]  
    y = data1['high_growth'] 
    
    logit_model = LogisticRegression()
    
    rmse_scores = np.sqrt(-cross_val_score(logit_model, X, y, scoring='neg_mean_squared_error', cv=5))
    
    mean_rmse = np.mean(rmse_scores)
    
    rmse_list.append({'Model': model_name, 'RMSE': mean_rmse})

rmse_table = pd.DataFrame(rmse_list)

print(rmse_table)

    Model      RMSE
0  model1  0.341814
1  model2  0.333976
2  model3  0.331994


## CART

In [36]:
results = []

for model_name, features in zip(['model1', 'model2', 'model3'], [model1, model2, model3]):
    X = data1[features]
    y = data1['high_growth']
    
    cart_model = DecisionTreeClassifier()
    
    # Calculate AUC
    auc_scores = cross_val_score(cart_model, X, y, scoring='roc_auc', cv=5)
    mean_auc = np.mean(auc_scores)
    
    # Calculate RMSE
    rmse_scores = np.sqrt(-cross_val_score(cart_model, X, y, scoring='neg_mean_squared_error', cv=5))
    mean_rmse = np.mean(rmse_scores)
    
    results.append({'Model': model_name, 'AUC': mean_auc, 'RMSE': mean_rmse})

results_table = pd.DataFrame(results)

print(results_table)

    Model       AUC      RMSE
0  model1  0.953288  0.199131
1  model2  0.964310  0.167552
2  model3  0.976047  0.136250


## RANDOM FOREST

In [25]:
data_train, data_holdout = train_test_split(data1, train_size=0.8, random_state=42)

In [34]:

grid = {
    "max_features": [5, 6, 7],
    "criterion": ["gini"],
    "min_samples_split": [11, 16],
}

# Assuming 'model3' is the feature you want to use
features = [model3]

X = data1.drop(columns=['high_growth'])
y = data1['high_growth']

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rf_classifier = RandomForestClassifier()

# Lists to store results
model_idx = []
auc_scores = []
rmse_scores = []

for idx, feature_set in enumerate(features, start=1):
    X_train_model = X_train[feature_set]
    
    # Create GridSearchCV
    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=grid, cv=5, scoring='roc_auc')
    
    # Fit the model using grid search
    grid_search.fit(X_train_model, y_train)
    
    print(f"Model {idx} Best Parameters:", grid_search.best_params_)
    
    # Get the best model from grid search
    best_model = grid_search.best_estimator_
    
    # Calculate ROC AUC on the holdout set
    X_holdout_model = X_holdout[feature_set]
    y_pred_proba = best_model.predict_proba(X_holdout_model)[:, 1]
    auc = roc_auc_score(y_holdout, y_pred_proba)
    print(f"Model {idx} ROC AUC on Holdout Set:", auc)
    
    # Calculate RMSE using cross-validation
    scores = cross_val_score(best_model, X_train_model, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = (-scores.mean()) ** 0.5
    print(f"Model {idx} RMSE CV:", rmse)
    
    # Append results to lists
    model_idx.append(f"Model {idx}")
    auc_scores.append(auc)
    rmse_scores.append(rmse)

# Create a DataFrame to display results as a table
results_table = pd.DataFrame({
    'Model': model_idx,
    'AUC': auc_scores,
    'RMSE': rmse_scores
})

print("\nTable of Results:")
print(results_table)


Model 1 Best Parameters: {'criterion': 'gini', 'max_features': 7, 'min_samples_split': 16}
Model 1 ROC AUC on Holdout Set: 0.9968059724495129
Model 1 RMSE CV: 0.17212454742550576

Table of Results:
     Model       AUC      RMSE
0  Model 1  0.996806  0.172125
